# <center>Entrenar y evaluar una red neuronal convolucional para resolver un problema de clasificación de imágenes</center>

<div style="background-color: #fdebd0 ">
<b>Este TP tiene 3 objetivos:</b><br>
- Describir conceptos vinculados al entrenamiento y uso de <i>ConvNets</i> para la clasificación de imágenes<br>
- Familiarizarse con la librería PyTorch<br>
- Reproducir un protocolo para entrenar una red neuronal convolucional y clasificar imágenes.</div>


PyTorch es una librería de aprendizaje automático de código abierto para Python, desarrollada principalmente por el grupo de investigación de inteligencia artificial de Facebook. Dentro de PyTorch, el paquete <code>torchvision</code> consiste en conjuntos de datos populares, arquitecturas de modelos y transformaciones de imágenes comunes para la visión artificial.

En la primera parte del trabajo práctico, utilizaremos el conjunto de datos CIFAR10 disponible en <code>torchvision</code>.  Este dataset sirve para aprender a resolver un problema de clasificación con 10 clases: 'avión', 'automóvil', 'pájaro', 'gato', 'ciervo', 'perro', 'rana', 'caballo', 'barco', 'camión'. Las imágenes en CIFAR-10 son de tamaño 3x32x32, es decir, imágenes en color de 3 canales de 32x32 píxeles de tamaño: https://www.cs.toronto.edu/~kriz/cifar.html

El código está basado en el tutorial de PyTorch siguiente: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

<img src="cifar10.png"></img>

Para resolver el problema de clasificación de imágenes, seguiremos los siguientes pasos:

- Cargar y normalizar los conjuntos de datos de entrenamiento y pruebas utilizando CIFAR10.
- Configurar una red neuronal de convolución
- Definir una función de pérdida
- Optimizar la red sobre los datos de entrenamiento
- Probar el rendimiento de la red con los datos de test

Por cada paso, se solicita responder a una serie de preguntas.

## 1. Cargar y normalizar el dataset CIFAR10

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

#The output of torchvision datasets are PILImage images of range [0, 1]. 
#We transform them to Tensors of normalized range [-1, 1].

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


<div style="background-color: #fdebd0 ">
<b>Preguntas</b><br>
1) ¿Cuál es el tamaño del dataset de entrenamiento y del dataset de test? (Imprimir el resultado con la función <code>print</code>)<br>
2) ¿Por qué el parametro <code>shuffle</code> se configura "True" para el dataset de entrenamiento y "False" para el dataset de test? ¿De qué sirve este parametro?<br>


<div style="background-color:#45b39d ">Respuestas:<br>1)El set de entrenamiento tiene un tamaño de 50000 datos y el set de test tieneun tamaño de 10000 datos.<br>2)Shuffle sirve para reordenar aleatoriamente el dataset en cada iteración. En este caso se ordenan el set de datos de entrenamiento porque es el que se requiere ir comparando cada uno.</div>

In [2]:
print (len(trainset))
print (len(testset))

50000
10000


El código siguiente permite mostrar imagenes aleatorias del dataset de entrenamiento y su etiqueta real:

In [3]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def imagesFromBatches(iterator,quantity):
    dataiter = iter(iterator)
    images, labels = dataiter.next()    
    imshow(torchvision.utils.make_grid(images))
    print(' '.join('%5s' % classes[labels[j]] for j in range(quantity)))
    return (images,labels)

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

<Figure size 640x480 with 1 Axes>

horse truck  frog  ship


## 2. Configurar la CNN

In [4]:
import torch.nn as nn
import torch.nn.functional as F

from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

summary(net,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.31
----------------------------------------------------------------


<div style="background-color: #fdebd0 ">
<b>Preguntas</b><br>
1) ¿Cuántas capas tiene esta ConvNet?<br>
2) Explicar los parametros de cada capa. ¿Cuántos filtros se utilizan en las capas de convolución? ¿Cuál es el tamaño de los filtros? <br>
3) ¿Cuál es la diferencia entre la función <code>init</code> y <code>forward</code>?<br>
4) ¿De qué sirve la función view()? Explicar sus parametros.<br>
5) ¿Cuántos paramétros en total se tiene que aprender con esta ConvNet? <br>
6) ¿Por qué se utiliza la función <code>conv2d</code> aunque tenemos imagenes con 3 canales?
</div>

<div style="background-color:#45b39d ">Respuestas:<br>1)Por lo que se ve en el algoritmo hay 2 capas de convolución y 1 capa de pooling, es decir en total hay 3 capas ocultas (hidden)<br>2)conv1: con entrada 3 canales ya que es una imagen RGB, tamaño 6 kernels para la salida y un tamaño 5x5 que corresponde al número de filas y columnas que tendrá de tamaño cada uno de los kernels utilizados. conv2 tiene como una entrada 6 canales, 16 kernels de tamaño 5x5. Y por ultimo entre estas dos convoluciones tenemos un maxpool con stride 2. <br>3)En el init se instancian las distintas capas y se le asignan variables. En el forward se ejecutan las capas, ya que este recive lo anteriormente declarado<br>4) La función view devuelve un tensor con los mismos datos que el metodo self (lo que significa que el tensor devuelto tiene el mismo número de elementos), pero con una forma diferente. Para este caso sus parametros son -1 es por defecto para el numero de filas y el 16*5*5 corresponde al total de elementos<br>5)Se tienen que aprender 62006 parametros<br>6)Porque convsd se refiere a que son imagenes en dos dimensiones y los canales hacen referencia a que es una imagen RGB, con tres canales de colores</div>

## 3. Entrenar la CNN

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

BrokenPipeError: [Errno 32] Broken pipe

- Ver la video: https://www.youtube.com/watch?v=ErfnhcEV1O8 - A Short Introduction to Entropy, Cross-Entropy and KL-Divergence

- Leer: http://ruder.io/optimizing-gradient-descent/index.html - An overview of gradient descent optimization algorithm

<div style="background-color: #fdebd0 ">
<b>Preguntas</b><br>
1) ¿Qué hace la función <code>CrossEntropyLoss</code>? Qué devuelve? Con qué otra función se podría reemplazar <code>CrossEntropy</code>?<br>
2) ¿Cuál es la diferencia principal entre los métodos de optimización Gradient Descent, Stochastic Gradient Descent y Mini-Batch Gradient Descent?<br>
3) ¿En nuestro ejemplo, qué método utilizamos? En qué parte del código se podría cambiar el tamaño del batch?<br>
4) ¿Qué metafora podemos utilizar para entender la idea del parametro <code>momentum</code>?<br>
5) ¿Podría ser útil aumentar el número de epoch? ¿Por qué? De qué sirve este parametro?

</div>

<div style="background-color:#45b39d ">Respuestas:<br>1)CrossEntropyLoss se utiliza para la clasificación multiclase, es decir, predice una de varias clases para cada ejemplo. Puede ser reemplazado por MultiLabelMarginLoss para la clasificación multi-etiqueta<br>2)SGD converge más rápido en comparación con GD ya que GD, tiene que recorrer todas las muestras del set de entrenamiento, en SGD, se usa uno o un subset del set de entrenamiento. Si se utiliza un subset, se denomina Mini-Batch Gradient Descent.<br>3)En el ejemplo utilizamos Stochastic Gradient Descent. El tamaño del batch se puede cambiar cambiando el valor al momento de definir trainloader<br>4) Momemtum puede enternderse como el impuso para una converencia más rápida, Es decir para disminuir la tasa de aprendizaje se debe utilizar un impulso alto. Metaforicamente es como una patadita para acelar el aprendizaje<br>5)El número del parametro epoch es un hiperparámetro de pendiente de gradiente que controla el número de pasadas completas a través del set de datos de entrenamiento, se compone de uno o más batch. Si lo aumentamos hacemos que el error del modelo se minimise</div>

## 4. Evaluar la CNN

Tomamos 4 ejemplos del dataset de test:

In [ ]:
BatchSize=4

images,labels= imagesFromBatches(testloader,BatchSize)

Clasificamos estas 4 imágenes con nuestra CNN: 

In [ ]:
outputs = net(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

<div style="background-color: #fdebd0 ">
<b>Pregunta</b><br>
1) ¿Cuál es el rendimiento del modelo entrenado sobre estos primeros ejemplos?

</div>

Calculamos el rendimiento de nuestra CNN sobre todos los datos del training set:

<div style="background-color:#45b39d ">Respuesta:<br>El redimiento para las primeras 10000 imagenes es de un 60% </div>

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Miramos el rendimiento de la CNN calculando su exactitud según cada etiqueta:

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

<div style="background-color: #fdebd0 ">
<b>Preguntas:</b><br>
1) ¿Qué paramétros podrían modificar para tratar de mejorar el rendimiento de la CNN? <br>
2) ¿Cómo se llama la arquitectura de CNN que hemos utilizado? (ver slides del curso y https://medium.com/@sidereal/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5 y https://adeshpande3.github.io/The-9-Deep-Learning-Papers-You-Need-To-Know-About.html)<br>
3) ¿Qué pasa si tratan de agregar más filtros en la segunda capa de convolución? <br>
4) ¿Qué pasa si trata de agregar una tercera capa de convolución y pooling? <br>
5) ¿En la literatura, qué arquitecturas CNN permiten obtener mejores rendimiento que la arquitectura LeNet-5? Cuál es el limite de estas arquitecturas? <br>
</div>

<div style="background-color:#45b39d ">Respuestas:<br>1)Podriamos aumentar las capas de convolución, los filtros o la cantidad de fc. Pero se debe  experimentar.O tambien se puede aumentar las epocas<br>2)La arquitectura utilizada es LeNet-5<br>3)Si agregamos más filtros podemos aprender otras cosas como texturas, llegando a reconocer el objeto en sí<br>4)Se mejoraría el algoritmo<br>5)</div>

<div style="background-color: #fdebd0 ">
<b>T.P</b><br>
Optimizar una CNN para resolver el problema asociado al dataset Fashion-MNIST (https://pytorch.org/docs/stable/torchvision/datasets.html#fashion-mnist). <br>

1) ¿La arquitectura LeNet-5 es mejor que Random Forest? Comparar el rendimiento obtenido con lo obtenido por el algoritmo RandomForest (con 50 estimadores).
</div>